EXTRA: Se importan los dataframes en Power BI:

- Se importan las librerías correspondientes:

    - Pymysql y cryptography para conectarse a la base de datos de MySql
    - Pandas para guardar y manipular los data frames

- Se crea la conexion con el servidor y la base de datos
- Se mete cada tabla de la base de datos en un data frame
- Se manipulan las variables numéricas de cada dataframe para formatearlas como "str" y cambiar los "." por "," (Power BI trabaja con comas en lugar de puntos).

    *También se puede cambiar la manera de leer los decimales en un archivo d epower BI pero suele fallar bastante

In [ ]:
import pymysql
import cryptography
import pandas as pd

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='4321',
                             database='business_db')

transactionsDf = pd.read_sql_query('select * from transactions', connection)

transactionsDf['amount'] = transactionsDf['amount'].astype(str)
transactionsDf['amount'] = transactionsDf['amount'].str.replace('.', ',')
transactionsDf['lat'] = transactionsDf['lat'].astype(str)
transactionsDf['lat'] = transactionsDf['lat'].str.replace('.', ',')
transactionsDf['longitude'] = transactionsDf['longitude'].astype(str)
transactionsDf['longitude'] = transactionsDf['longitude'].str.replace('.', ',')

companiesDf = pd.read_sql_query('select * from companies', connection)
creditCardsDf = pd.read_sql_query('select * from credit_cards', connection)
productsDf = pd.read_sql_query('select * from products', connection)

productsDf['weight'] = productsDf['weight'].astype(str)
productsDf['weight'] = productsDf['weight'].str.replace('.', ',')
productsDf['price_dollars'] = productsDf['price_dollars'].astype(str)
productsDf['price_dollars'] = productsDf['price_dollars'].str.replace('.', ',')

usersDf = pd.read_sql_query('select * from users', connection)
boughtProductsDf = pd.read_sql_query('select * from bought_products', connection)

Posteriormente se formatean los tipos de las columnas (Int, float, etc. Según sea necesario) y se establecen las conexiones entre las tablas (creadas previamente usando los dataframes), usando las keys correspondientes.

![Conexiones Power BI](A_Conexiones.png)

Con todo seteado para trabajar en el informe, se procede ahora a mostrar cada gráfico de manera individual, las partes que lo componen y el código subyacente.

NIVEL 1

EJERCICIO 1: "Distribución del Amount por Venta"

Corresponde al Ejercicio 1 del Nivel 1 (8.1) ("Una Variable Numérica").

Campos seleccionados:
- transactionsDf.amount

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

![Nivel 1 / Ejercicio 1](A_LVL1E1.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')

sns.catplot(dataset['amount'], 
            kind = 'box')
plt.xlabel('Amount ($)')

plt.show()

EJERCICIO 2: "Relación del Peso del Producto y su Precio"

Corresponde al Ejercicio 2 del Nivel 1 (8.1) ("Dos Variable Numéricas").

Campos seleccionados:
- productsDf.weight
- productsDf.price_dollars

![Nivel 1 / Ejercicio 2](A_LVL1E2.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(weight, price_dollars)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')

sns.relplot(dataset, 
            x= 'weight', y= 'price_dollars', 
            kind='line', 
            errorbar= 'sd')
plt.xlabel('Peso (Kg)')
plt.ylabel('Precio ($)')

plt.show()

EJERCICIO 3: "Ratio de Venta por Producto"

Corresponde al Ejercicio 3 del Nivel 1 (8.1) ("Una Variable Categórica").

Campos seleccionados:
- productsDf.product_name
- boughtProductsDf.product_id
- boughtProductsDf.transactions_id
- boughtProductsDf.index*

*Esta columna (columna de index por default de Power Query) se ha creado porque los scripts de Python en Power BI eliminan las filas duplicadas. En esta tabla puente (Bought Products), hay pedidos que repiten productos en  alguna ocasión. Sin este index los datos quedarían incompletos

![Nivel 1 / Ejercicio 3](A_LVL1E3.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(product_name, id, id.1)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import pandas as pd
import plotly.express as px

dataset = dataset.groupby('product_id').agg({'transactions_id': 'count', 'product_name': 'first', 'index':'first'}).reset_index()
dataset = dataset.drop(columns='index')

cpvPie = px.pie(dataset, 
                values='transactions_id',
                names='product_name',
                color_discrete_sequence=px.colors.sequential.ice)

cpvPie.update_traces(insidetextorientation='radial')

cpvPie.update_layout(
    margin = dict(t=50, l=100, r=10, b=50))

cpvPie.write_image("pie_products.png")

EJERCICIO 4: "Ingresos por User Cohort"

Corresponde al Ejercicio 4 del Nivel 1 (8.1) ("Una Variable Categórica y una Numérica").

Campos seleccionados:
- transactionsDf.amount
- usersDf.user_cohot

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

![Nivel 1 / Ejercicio 4](A_LVL1E4.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, user_cohort)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import plotly.express as px

cpvPie = px.pie(dataset, 
                values='amount', 
                names='user_cohort', 
                color_discrete_sequence=px.colors.sequential.Magma_r)

cpvPie.update_layout(
    margin = dict(t=0, l=0, r=0, b=0))

cpvPie.write_image("amount_per_cohort.png")

EJERCICIO 5: "Dirección de las Transacciones Realizadas"

Corresponde al Ejercicio 5 del Nivel 1 (8.1) ("Dos Variables Categóricas").

Campos seleccionados:
- transactionsDf.id
- usersDf.country_user
- companiesDf.country

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

![Nivel 1 / Ejercicio 5](A_LVL1E5.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country_user, country, id)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

sns.displot(dataset, 
            x='country', y='country_user', 
            height=4, aspect=4.75, 
            cbar= True,
            col_order = ['Canada', 'United Kingdom', 'United States'])

plt.show()

EJERCICIO 6: "Ingresos por País y Empresa"

Corresponde al Ejercicio 6 del Nivel 1 (8.1) ("Tres Variables").

Campos seleccionados:
- transactionsDf.amount
- companiesDf.country
- companiesDf.company_name
- companiesDf.company_id

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

![Nivel 1 / Ejercicio 6](A_LVL1E6.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, company_name, country)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import pandas as pd
import plotly.express as px

dataset = dataset.groupby('company_id').agg({'amount': 'sum', 'company_name':'first', 'country':'first'}).reset_index()

# Crea gráfico sunburst
fig = px.sunburst(dataset,
                  path=['country', 'company_name'], 
                  values='amount',
                  width=1000,
                  height=1000)

fig.update_traces(insidetextorientation='radial')

fig.update_layout(
    margin = dict(t=0, l=0, r=0, b=0))

fig.write_image("sunburst.png")

EJERCICIO 7: "Contenido del Pedido por Transacción"

Corresponde al Ejercicio 7 del Nivel 1 (8.1) ("Graficar un Pairplot").

Campos seleccionados:
- transactionsDf.id
- transactionsDf.amount
- transactionsDf.cantidad_productos
- productsDf.weight
- boughtProductsDf.index

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

![Nivel 1 / Ejercicio 7](A_LVL1E7.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(id, amount, weight, cantidad_productos)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

dataset = dataset.groupby('id').agg({'amount':'sum', 'weight':'sum', 'cantidad_productos':'first', 'index':'first'}).reset_index().rename(columns={'amount':'amount ($)', 'weight':'weight (Kg)'})

dataset = dataset.drop(columns='index')

grid = sns.PairGrid(dataset, 
                    diag_sharey=False)
grid.map_diag(sns.kdeplot, 
              fill=True)
grid.map_lower(sns.scatterplot, 
               alpha=0.5)
grid.map_upper(sns.kdeplot)

plt.show()

NIVEL 2

EJERCICIO 1: "Correlación de Todas las Variable Numéricas"

Corresponde al Ejercicio 1 del Nivel 2 (8.1) ("Correlación de todas las variables numéricas").

Campos seleccionados:
- transactionsDf.amount
- transactionsDf.lat
- transactionsDf.longitude
- transactionsDf.cantidad_productos
- productsDf.price_dollars
- productsDf.weight
- boughtProductsDf.index
- transactionsDf.id

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

![Nivel 2 / Ejercicio 1](A_LVL2E1.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, lat, longitude, cantidad_productos, price_dollars, weight)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

todoNumerico = dataset.groupby('id').agg({'amount':'first', 'cantidad_productos':'first', 'price_dollars':'sum', 'weight':'sum','lat':'first', 'longitude':'first', 'index':'first'}).reset_index().rename(columns={'amount':'amount ($)', 'price_dollars':'price ($)', 'weight':'weight (Kg)'})
todoNumerico = todoNumerico.drop(columns=['id','index'])
todoNumericoCorr = todoNumerico.corr()


minValues = todoNumericoCorr.min() # Min de cada columna en una nueva columna
minValues = minValues.min() # Min de todos los min

fig, ax = plt.subplots(figsize=(10,10))    
sns.heatmap(todoNumericoCorr, 
            vmin=minValues, 
            annot=True, 
            linewidths= 0.5, 
            cmap='crest', 
            square=True)

plt.show()

EJERCICIO 2: "Distribución y Relación de la Cantidad de Productos  Según la Media de Amount"

Corresponde al Ejercicio 2 del Nivel 2 (8.1) ("Implementa un jointplot").

Campos seleccionados:
- transactionsDf.amount
- transactionsDf.cantidad_productos
- transactionsDf.declined

![Nivel 2 / Ejercicio 2](A_LVL2E2.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, cantidad_productos)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

ticks = dataset['cantidad_productos'].sort_values(ascending=True).unique() #Si no crea columnas x.5 para cada cantidad


relPlotNum = sns.JointGrid(dataset, 
                           y='amount', x='cantidad_productos', 
                           hue='declined',
                           height= 8 )
relPlotNum.plot_joint(sns.kdeplot)
relPlotNum.plot_marginals(sns.histplot, 
                          kde=True)
relPlotNum.ax_joint.set_xticks(ticks)
relPlotNum.ax_joint.set_xlabel('Cantidad de Productos por Pedido')
relPlotNum.ax_joint.set_ylabel('Amount ($)')

plt.show()

NIVEL 3

EJERCICIO 1: "Distribución del Amount en las Transacciones por País y Tipo"

Corresponde al Ejercicio 1 del Nivel 3 (8.1) ("Implementa un violinplot combinado con otro tipo de gráfico").

Campos seleccionados:
- usersDf.country_user
- transactionsDf.amount
- transactionsDf.declined

![Nivel 3 / Ejercicio 1](A_LVL3E1.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(country_user, amount, declined)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

orderDf = dataset.groupby('country_user')['amount'].count().sort_values(ascending=False).index

base = sns.catplot(dataset, 
                   kind='violin', 
                   split=True, 
                   x='country_user', y='amount', 
                   hue='declined', 
                   inner='quart', 
                   order=orderDf, 
                   aspect=2)
sns.swarmplot(dataset, x='country_user', y='amount', hue= 'declined', ax=base.ax, palette='dark:white', size=4)

plt.show()

EJERCICIO 2: "Distribución de las Ventas por País y Hora"

Corresponde al Ejercicio 2 del Nivel 3 (8.1) ("Genera un FacetGrid para visualizar múltiples aspectos de datos simultáneamente").

Campos seleccionados:
- usersDf.country_user
- transactionsDf.id
- transactionsDf.hora*

Además, también se ha usado el campo transactionsDf.declined para filtrar por declined == 0

*Se ha creado una columna nueva en DAX donde aparece la "hora" del timestamp asociado a cada record

![Nivel 3 / Ejercicio 2](A_LVL3E2.png)

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(hora, country_user)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

orderDf = dataset.groupby('country_user')['id'].count().sort_values(ascending=False).index

facetOrderDf = dataset['hora'].sort_values(ascending=True)
facetOrderDf = facetOrderDf.unique()

g = sns.FacetGrid(dataset, 
                  col='country_user', 
                  col_order=orderDf,
                  height=5, aspect=1)
g.map(sns.countplot, 'hora', order=facetOrderDf)
g.set_axis_labels('Hora', 'Transacciones')
g.set_titles(col_template='{col_name}')
g.fig.suptitle('Ventas Distribuidas por Hora y Región', y=1.05, x=0.53)

plt.show()